In [ ]:
import math
import matplotlib.pyplot as plt
from numba import cuda, jit
import numpy as np
import openpmd_api as io
import scipy.constants
from scipy.ndimage import gaussian_filter
import time


## Things to do in the future

* specify a cutoff, behind all fields are written as 0 instead of a small value e.g. 1e-10 * field_max
* multi GPU / multi Node calculations  field size are limited by GPU memory size

## Things to note
* The Convolutional PML fields aren't changed, which might lead to unexpected simulations
* Only use this initialization on the 0 timestep (for now)

In [ ]:
# this cell is only required for testing the write proccess
try:
    del readSeries
    print("del readSeries")
except:
    pass
try:
    del writeSeries
    print("del writeSereies")
except:
    pass


## Set path to checkpoint

In [ ]:

# you can use one checkpoint explicitly checkpoint_0.h5
# or a time series with checkpoint_%T.h5
# the warning can be ignored when using the explicit checkpoint, we want just a single file
#path = "/bigdata/hplsim/scratch/spreng88/runs/bigFieldWithoutInit23/simOutput/checkpoints/checkpoint_0.h5"
readFile = "/bigdata/hplsim/scratch/spreng88/runs/old/restart/tmp_%T.h5"
outputFile = "/bigdata/hplsim/scratch/spreng88/runs/restart/write_%T.h5"
#readFile = "/bigdata/hplsim/production/wrobel45/PWFA-bunch-runs/033_vacuum_prop_small/simOutput/checkpoints/checkpoint_%T.h5"
#outputFile = "/bigdata/hplsim/scratch/spreng88/runs/restart/test_%T.h5"
cuda.select_device(0)
readSeries = io.Series(readFile, io.Access.read_only)
writeSeries = io.Series(outputFile, io.Access.create)

## General setup

In [ ]:
nth = 1             # only use every nth - particle (for faster calculation for tests)

# chunk_size set later for now
#chunk_size = [100, 100, 100]   # z, y, x

# add field to iteration 0
inputIteration = readSeries.iterations[0] 
outputIteration = writeSeries.iterations[0]

# Read attributes of the simulation that might be needed later
# Everything is calculated in PIConGPU units
# For SI units: length_si = value_pic * unit_length
cell_depth = inputIteration.get_attribute("cell_depth")          # z
cell_height = inputIteration.get_attribute("cell_height")        # y
cell_width = inputIteration.get_attribute("cell_width")          # x

unit_efield = inputIteration.get_attribute("unit_efield")
unit_bfield = inputIteration.get_attribute("unit_bfield")
unit_charge = inputIteration.get_attribute("unit_charge")
unit_mass = inputIteration.get_attribute("unit_mass")
unit_speed = inputIteration.get_attribute("unit_speed")
unit_length = inputIteration.get_attribute("unit_length")
unit_time = inputIteration.get_attribute("unit_time")

pi = scipy.constants.pi
c = scipy.constants.c / unit_speed
eps0 = inputIteration.get_attribute("eps0")
mue0 = inputIteration.get_attribute("mue0")

## Cuda functions

In [ ]:

@cuda.jit(device=True)
def particleV(px, py, pz, mass):
    """
    calculate particle speed from momentum
    """
    
    m2p2 = math.sqrt( (mass)**2 + px**2 + py**2 + pz**2)
    
    vx = px / m2p2 * c
    vy = py / m2p2 * c
    vz = pz / m2p2 * c
    
    return vx, vy, vz


@cuda.jit(device=True)
def param1(rx,ry,rz, rqx,rqy,rqz, vx,vy,vz):
    """
    calculate some parameters that could be reused
    r: position of field calculation
    rq: position of charge q at time t
    rq_tr: position of charge at retarded time tr
    """
    
    #
    # solution equation:
    # dt = t - tr   = time - retarded_time
    # r where we want the field
    # rq position of charge
    # |c*dt| = |r - (rq - v*dt)|
    # solve for dt:
    # => (c^2 - |v|^2) * dt^2 - 2(r*v - rq*v) * dt + 2 * r*rq - |r|^2 - |rq|^2
    #
    a = (c**2 - (vx**2 + vy**2 + vz**2))
    b = -2 * ((rx-rqx)*vx + (ry-rqy)*vy + (rz-rqz)*vz)
    d = 2 * (rx*rqx + ry*rqy + rz*rqz) - (rx**2 + ry**2 + rz**2) - (rqx**2 + rqy**2 + rqz**2)
    dt = (-b + math.sqrt(b**2 - 4*a*d)) / (2*a)
    
    rq_trx = rqx - vx * dt
    rq_try = rqy - vy * dt
    rq_trz = rqz - vz * dt

    dvx = rx - rq_trx
    dvy = ry - rq_try
    dvz = rz - rq_trz
    
    distance = math.sqrt(dvx**2 + dvy**2 + dvz**2)
    
    if distance == 0:
        distance = -1
    
    #n =  distanceVec / distance
    nx = dvx / distance
    ny = dvy / distance
    nz = dvz / distance
    
    return nx, ny, nz, distance, dvx, dvy, dvz #n, distance, distanceVec


@cuda.jit(device=True)
def retardedEFieldParallel(q, nx, ny, nz, distance, dvx, dvy, dvz, vx, vy, vz):
    """
    field for one position
    q: charge
    nx, ny, nz: unit vector to retarded position
    distance: distance to retarded position
    dvx, dvy, dvz: distance Vector to retarded position
    vx, vy, vz: velocity vector of the charge
    """
    
    if distance == -1:
        nx = 1
        ny = 0
        xz = 0
    
    factor = q / (4 * pi * eps0)
    
    ux = c * nx - vx
    uy = c * ny - vy
    uz = c * nz - vz
    

    scalar = factor * distance / (dvx*ux + dvy*uy + dvz*uz)**3 * (c**2 - (vx**2 + vy**2 + vz**2))
    
    return  scalar * ux, scalar * uy, scalar * uz


@cuda.jit(device=True)
def returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass):
    """calculate the fields for the position,
    currently ignoring the offset between the grid corner and field points
    because we will use smoothing in the end over the field anyway"""
    
    vx, vy, vz = particleV(px, py, pz, mass)
    
    ## E Field ------------------------------------------------------------
    nx, ny, nz, distancez, dvx, dvy, dvz = param1(rx, ry, rz, rqx, rqy, rqz, vx,vy,vz)
    ex, ey, ez = retardedEFieldParallel(q, nx, ny, nz, distancez, dvx, dvy, dvz , vx, vy, vz)

    
    ## B Field ------------------------------------------------------------
    bx = ( ny * ez - nz * ey ) / c
    by = ( nz * ex - nx * ez ) / c
    bz = ( nx * ey - ny * ex ) / c
    
    return ex, ey, ez, bx, by, bz

In [ ]:
@cuda.jit
def particleParallel(Ex,Ey,Ez, Bx,By,Bz, q_, rqx_,rqy_,rqz_, px_,py_,pz_, mass_, weighting_, xdim, ydim, zdim, particleCount, chunk_offset):
    """
    calculate all particles in parallel, loop over every point in the field
    
    Ex,Ey,Ez E field where new field is added
    Bx,By,Bz B -"-
    
    _ is on all input arrays read from checkpoint 
    q_ array from checkpoint with charges
    rq_ position data of particles
    p_ momentum of particles
    weighting_ macro particle weighting
    
    xdim, ydim, zdim length of dimension of Ex,Bx, ...
    Ex/Bx need to be 1D for atomic add
    """
    
    tix = cuda.threadIdx.x
    bix = cuda.blockIdx.x
    bdx = cuda.blockDim.x
    
    index = tix + bix * bdx    # particle index
    index = index
    
    if index < particleCount:
    
        q = q_[index] * weighting_[index]
        rqx = rqx_[index] * cell_width
        rqy = rqy_[index] * cell_height
        rqz = rqz_[index] * cell_depth
        px = px_[index]
        py = py_[index]
        pz = pz_[index]
        mass = mass_[index] * weighting_[index]

        for x in range(xdim):

            rx = (x + chunk_offset[2]) * cell_width
            rex = (x + chunk_offset[2] + 0.5) * cell_width

            for y in range(ydim):

                ry = (y + chunk_offset[1]) * cell_height
                rey = (y + chunk_offset[1] + 0.5) * cell_height

                for z in range(zdim):    
                    
                    rz = (z + chunk_offset[0]) * cell_depth
                    rez = (z + chunk_offset[0] + 0.5) * cell_depth

                    fieldIndex = x + y * xdim + z * xdim * ydim
                    
                    ex, ey, ez, bx, by, bz = returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass)
                    
                    cuda.atomic.add(Ex, fieldIndex, ex * nth)
                    cuda.atomic.add(Ey, fieldIndex, ey * nth)
                    cuda.atomic.add(Ez, fieldIndex, ez * nth)

                    cuda.atomic.add(Bx, fieldIndex, bx)
                    cuda.atomic.add(By, fieldIndex, by)
                    cuda.atomic.add(Bz, fieldIndex, bz)

In [ ]:
@cuda.jit
def FieldParallel(Ex,Ey,Ez, Bx,By,Bz, q_, rqx_,rqy_,rqz_, px_,py_,pz_, mass_, weighting_, xdim, ydim, zdim, particleCount, chunk_offset):
    """
    Calculate all field points in parallel, loop over all particles
    
    Ex,Ey,Ez E field where new field is added
    Bx,By,Bz B -"-
    
    a trailing _ is on all input arrays read from checkpoint 
    q_ array from checkpoint with charges
    rq_ position data of particles
    p_ momentum of particles
    
    xdim, ydim, zdim length of dimension of Ex,Bx, ...
    Ex/Bx need to be 1D for atomic add
    """
    tix = cuda.threadIdx.x
    bix = cuda.blockIdx.x
    bdx = cuda.blockDim.x
    
    fieldIndex = tix + bix * bdx    # field index
    
    # calculate 3d coordinate from 1d field coordinate array
    # fieldIndex = x + y * xdim + z * xdim * ydim
    z = math.floor( fieldIndex / (xdim*ydim) )
    y = math.floor( (fieldIndex - z * xdim*ydim) / xdim )
    x = fieldIndex - z * xdim*ydim - y * xdim
    
    x = x + chunk_offset[2]
    y = y + chunk_offset[1]
    z = z + chunk_offset[0]
    
    if fieldIndex < xdim*ydim*zdim:
        
        #for index in range(particleCount):
        # or add filter if you only want the field of some particles of that species
        for index in range(particleCount):
        
            q = q_[index] * weighting_[index]
            rqx = rqx_[index] * cell_width
            rqy = rqy_[index] * cell_height
            rqz = rqz_[index] * cell_depth
            px = px_[index]
            py = py_[index]
            pz = pz_[index]
            mass = mass_[index] * weighting_[index]
            
            rx = x * cell_width
            rex = (x + 0.5) * cell_width
            
            ry = y * cell_height
            rey = (y + 0.5) * cell_height
            
            rz = z * cell_depth
            rez = (z + 0.5) * cell_depth
            
            ex, ey, ez, bx, by, bz = returnFields(q, rx,rex,ry,rey,rz,rez, rqx,rqy,rqz, px,py,pz, mass)

            cuda.atomic.add(Ex, fieldIndex, ex * nth)
            cuda.atomic.add(Ey, fieldIndex, ey * nth)
            cuda.atomic.add(Ez, fieldIndex, ez * nth)

            cuda.atomic.add(Bx, fieldIndex, bx)
            cuda.atomic.add(By, fieldIndex, by)
            cuda.atomic.add(Bz, fieldIndex, bz)
            

In [ ]:
force_attribute_list = ['shape', 'position', 'axisLabels', 'gridGlobalOffset', 'gridSpacing']

def copy_attributes(inputAtr, outputAtr):
    dtypes = inputAtr.attribute_dtypes
    for attribute in inputAtr.attributes:
        attribute_value = inputAtr.get_attribute(attribute)
        if attribute in force_attribute_list:
            if type(attribute_value) != list:
                attribute_value = [attribute_value]
        outputAtr.set_attribute(attribute, attribute_value, dtypes[attribute])
        
def copy_series_attributes(inputAtr, outputAtr):
    attributes_to_skip = ['openPMD', 'openPMDextension']
    for attribute in inputAtr.attributes:
        if attribute in attributes_to_skip:
            continue
        attribute_value = inputAtr.get_attribute(attribute)
        outputAtr.set_attribute(attribute, attribute_value)
        
def print_attributes(atr):
    try:
        dtypes = atr.attribute_dtypes
        for attribute in atr.attributes:
            print(attribute, ":", atr.get_attribute(attribute), " - ", dtypes[attribute])
        print()
    except:
        print('errro in print attributes')

        
        
#### write mesh data ####

def copy_mesh_container(input_mesh_container, output_mesh_container, exclude_mesh=[]):
    copy_attributes(input_mesh_container, output_mesh_container)
    
    for mesh in input_mesh_container:
        if mesh in exclude_mesh:
            print("don't copy mesh:", mesh)
            continue
        print('mesh:', mesh)
        
        input_mesh = input_mesh_container[mesh]
        output_mesh = output_mesh_container[mesh]
        
        copy_mesh(input_mesh, output_mesh)
        
        writeSeries.flush()
        
    
def copy_mesh(input_mesh, output_mesh):
    copy_attributes(input_mesh, output_mesh)
    
    for mesh_record_component in input_mesh:
        print('  mesh_record_component:', mesh_record_component)
        
        input_mesh_record_component = input_mesh[mesh_record_component]
        output_mesh_record_component = output_mesh[mesh_record_component]
        
        copy_mesh_record_component(input_mesh_record_component, output_mesh_record_component)
        
    
def copy_mesh_record_component(input_mesh_record_component, output_mesh_record_component):
    copy_attributes(input_mesh_record_component, output_mesh_record_component)
    
    data_dtype = input_mesh_record_component.dtype
    data_shape = input_mesh_record_component.shape
    
    dataset = io.Dataset(data_dtype, data_shape)
    output_mesh_record_component.reset_dataset(dataset)
    
    data = input_mesh_record_component.load_chunk()
    readSeries.flush()
    
    output_mesh_record_component.store_chunk(data)
    
    
    
#### write particle data ####

def copy_particle_container(input_particle_container, output_particle_container):
    copy_attributes(input_particle_container, output_particle_container)
    
    for particle_species in input_particle_container:
        print('species:', particle_species)
        
        input_particle_species = input_particle_container[particle_species]
        output_particle_species = output_particle_container[particle_species]
        
        copy_particle_species(input_particle_species, output_particle_species)

        input_particle_patches = input_particle_species.particle_patches
        output_particle_patches = output_particle_species.particle_patches
        
        copy_particle_patches(input_particle_patches, output_particle_patches)
    
    
def copy_particle_species(input_particle_species, output_particle_species):
    copy_attributes(input_particle_species, output_particle_species)
    
    for record in input_particle_species:
        print('  record:', record)
        
        input_record = input_particle_species[record]
        output_record = output_particle_species[record]
        
        copy_record(input_record, output_record)
        
        writeSeries.flush()

        
def copy_record(input_record, output_record):
    copy_attributes(input_record, output_record)
    
    for record_component in input_record:
        print('    record_component:', record_component)
        
        input_record_component = input_record[record_component]
        output_record_component = output_record[record_component]
        
        copy_record_component(input_record_component, output_record_component)
    

def copy_record_component(input_record_component, output_record_component):
    copy_attributes(input_record_component, output_record_component)
    
    is_constant = input_record_component.constant
    
    if is_constant:
        value = input_record_component.get_attribute('value')
        output_record_component.make_constant(value)
        
        return
    
    data_dtype = input_record_component.dtype
    data_shape = input_record_component.shape
    
    dataset = io.Dataset(data_dtype, data_shape)
    output_record_component.reset_dataset(dataset)
    
    data = input_record_component.load_chunk()
    readSeries.flush()
    
    output_record_component.store_chunk(data)
    


#### write particle patch data ####

def copy_particle_patches(input_particle_patches, output_particle_patches):
    copy_attributes(input_particle_patches, output_particle_patches)
    
    for patch_record in input_particle_patches:
        print('  patch_record:', patch_record)
        
        input_patch_record = input_particle_patches[patch_record]
        output_patch_record = output_particle_patches[patch_record]
        
        copy_patch_record(input_patch_record, output_patch_record)
        
    writeSeries.flush()


def copy_patch_record(input_patch_record, output_patch_record):
    copy_attributes(input_patch_record, output_patch_record)
    
    for patch_record_component in input_patch_record:
        print('    patch_record_component:', patch_record_component)
        
        input_patch_record_component = input_patch_record[patch_record_component]
        output_patch_record_component = output_patch_record[patch_record_component]
        
        copy_patch_record_component(input_patch_record_component, output_patch_record_component)
        

def copy_patch_record_component(input_patch_record_component, output_patch_record_component):
    copy_attributes(input_patch_record_component, output_patch_record_component)
    
    data_dtype = input_patch_record_component.dtype
    data_shape = input_patch_record_component.shape
    
    dataset = io.Dataset(data_dtype, data_shape)
    output_patch_record_component.reset_dataset(dataset)
    
    data = input_patch_record_component.load()
    readSeries.flush()
    
    
    for i in range(np.prod(data_shape)):
        print('  ',i)
        output_patch_record_component.store(i, data[i])



        
def write():
    
    global writeSeries
    
    inputIteration
    
    # copy particle data
    input_particle_container = inputIteration.particles
    output_particle_container = outputIteration.particles
    
    copy_particle_container(input_particle_container, output_particle_container)
    
    
    # copy/write field data
    input_mesh_container = inputIteration.meshes
    output_mesh_container = outputIteration.meshes
    
    copy_mesh_container(input_mesh_container, output_mesh_container, exclude_mesh=['E', 'B'])
    
    del writeSeries
    
    
def write_general_data():

    # copy general attributes ----------------------------------
    copy_series_attributes(readSeries, writeSeries)

    copy_attributes(inputIteration, outputIteration)

In [ ]:
    
write_general_data()


In [ ]:
def smoothFields(ex, ey, ez, bx, by, bz, sigma):
    ex = gaussian_filter(ex.reshape(zdim, ydim, xdim), sigma)
    ey = gaussian_filter(ey.reshape(zdim, ydim, xdim), sigma)
    ez = gaussian_filter(ez.reshape(zdim, ydim, xdim), sigma)
    bx = gaussian_filter(bx.reshape(zdim, ydim, xdim), sigma)
    by = gaussian_filter(by.reshape(zdim, ydim, xdim), sigma)
    bz = gaussian_filter(bz.reshape(zdim, ydim, xdim), sigma)

## load particle data from the checkpoint
define the species identifier string, as in the simulation

In [ ]:
species = "b"

xpos_incell = inputIteration.particles[species]["position"]["x"][:]
ypos_incell = inputIteration.particles[species]["position"]["y"][:]
zpos_incell = inputIteration.particles[species]["position"]["z"][:]
xpos_offset = inputIteration.particles[species]["positionOffset"]["x"][:]
ypos_offset = inputIteration.particles[species]["positionOffset"]["y"][:]
zpos_offset = inputIteration.particles[species]["positionOffset"]["z"][:]
momentumx = inputIteration.particles[species]["momentum"]["x"][:]
momentumy = inputIteration.particles[species]["momentum"]["y"][:]
momentumz = inputIteration.particles[species]["momentum"]["z"][:]
weightings = inputIteration.particles[species]["weighting"][io.Record_Component.SCALAR][:]
charge = inputIteration.particles[species]["charge"][io.Record_Component.SCALAR][:]
mass = inputIteration.particles[species]["mass"][io.Record_Component.SCALAR][:]

readSeries.flush()

xpos = xpos_incell + np.float32(xpos_offset)
ypos = ypos_incell + np.float32(ypos_offset)
zpos = zpos_incell + np.float32(zpos_offset)

# free some memory
del xpos_incell, ypos_incell, zpos_incell
del xpos_offset, ypos_offset, zpos_offset


# use just every nth particle
# needs to be in a new numpy array
# because it must be continous in memory to be copied to gpu
momentumx = np.array(momentumx[::nth])
momentumy = np.array(momentumy[::nth])
momentumz = np.array(momentumz[::nth])
weightings = np.array(weightings[::nth])
charge = np.array(charge[::nth])
mass = np.array(mass[::nth])
xpos = np.array(xpos[::nth])
ypos = np.array(ypos[::nth])
zpos = np.array(zpos[::nth])
particleCount = len(mass)

## output to check particleCount and gridsize

In [ ]:
print("particleCount:", particleCount)
input_field_shape = inputIteration.meshes["E"]["x"].shape
xdim_total = input_field_shape[2]
ydim_total = input_field_shape[1]
zdim_total = input_field_shape[0]
input_field_shape_total = xdim_total * ydim_total * zdim_total
print("gridSize (z,y,x, total):", input_field_shape, input_field_shape_total)

field_dtype = inputIteration.meshes['E']['x'].dtype
field_dataset = io.Dataset( field_dtype, input_field_shape)

for field_name in ['E', 'B']:
    copy_attributes(inputIteration.meshes[field_name], outputIteration.meshes[field_name])
    for component in ['x', 'y', 'z']:
        copy_attributes(inputIteration.meshes[field_name][component], outputIteration.meshes[field_name][component])
        outputIteration.meshes[field_name][component].reset_dataset(field_dataset)

## calculate the fields

set the chunk_size to the number of cells if the gpu has enough memory

In [ ]:
chunk_size = np.array([768, 512, 768], dtype=np.int32)  # z y x
xdim = chunk_size[2]
ydim = chunk_size[1]
zdim = chunk_size[0]
shape = xdim * ydim * zdim

min_memory_device = shape * 24 + particleCount * 56
print("device needs a minimum memory of:",min_memory_device, "Bytes = {:.5} GB".format(min_memory_device*1e-9))

chunk_offset = np.array([0, 200, 0], dtype=np.int32)          # offset for chunk (z, y, z)


In [ ]:

if particleCount <= shape:    # maybe add a constant, but needs testing for optimal choise
    # calculate every field position in parallel (loop over the particles)
    # few particles, many field positions
    blockdim = 256                                              # number of threads per block (multiple 32 for optimal speed)
    griddim = math.ceil(shape / blockdim)                       # number of blocks in the grid
    cudaFunc = FieldParallel
    print("Fields in parallel")
    
else:
    # calculate every particle in parallel (loop over the entire field)
    # many particles, few field positions
    blockdim = 32  #256                                         # number of threads per block (multiple 32 for optimal speed)
    griddim = math.ceil(particleCount / blockdim)         # number of blocks in the grid
    cudaFunc = particleParallel
    print("particles in parallel")
    
print("particles to be processed:", particleCount)

# for test on a smaller number of particles
#blockdim = 1
#griddim = 1

array_dtype = np.float32                            # dtype for arrays
ex = np.zeros(shape, dtype=array_dtype)
ey = np.zeros(shape, dtype=array_dtype)
ez = np.zeros(shape, dtype=array_dtype)
bx = np.zeros(shape, dtype=array_dtype)
by = np.zeros(shape, dtype=array_dtype)
bz = np.zeros(shape, dtype=array_dtype)

fields = {}
fields['E'] = {}
fields['B'] = {}
fields['E']['x'] = ex
fields['E']['y'] = ey
fields['E']['z'] = ez
fields['B']['x'] = bx
fields['B']['y'] = by
fields['B']['z'] = bz

# All variables with _d at the and are for device variables
print("allocate/copy field/data arrays to device")
ex_d = cuda.to_device(ex)
ey_d = cuda.to_device(ey)
ez_d = cuda.to_device(ez)
bx_d = cuda.to_device(bx)
by_d = cuda.to_device(by)
bz_d = cuda.to_device(bz)

charge_d = cuda.to_device(charge)
xpos_d = cuda.to_device(xpos)
ypos_d = cuda.to_device(ypos)
zpos_d = cuda.to_device(zpos)
momentumx_d = cuda.to_device(momentumx)
momentumy_d = cuda.to_device(momentumy)
momentumz_d = cuda.to_device(momentumz)
mass_d = cuda.to_device(mass)
weightings_d = cuda.to_device(weightings)
chunk_offset_d = cuda.to_device(chunk_offset)

print("\nstart of field calculation time:", time.ctime())
starttime = time.time()

cudaFunc[griddim, blockdim](ex_d, ey_d, ez_d, bx_d, by_d, bz_d, charge_d, xpos_d, ypos_d, zpos_d, momentumx_d, momentumy_d, momentumz_d, mass_d, weightings_d, xdim, ydim, zdim, particleCount, chunk_offset_d)
cuda.synchronize()

exeTime = time.time()-starttime
print("time: {:.5}".format( exeTime ), "s")
print("avgTime per particle per cell:", exeTime / particleCount / (xdim*ydim*zdim), "s/(particle*cell)")


print("\nget field arrays from device", time.ctime())
ex_d.copy_to_host(ex)
ey_d.copy_to_host(ey)
ez_d.copy_to_host(ez)
bx_d.copy_to_host(bx)
by_d.copy_to_host(by)
bz_d.copy_to_host(bz)
print("field arrays copied from device to host", time.ctime())

# delete device variables, no longer needed after the parallel computation
del charge_d
del xpos_d
del ypos_d
del zpos_d
del momentumx_d
del momentumy_d
del momentumz_d
del mass_d
del weightings_d

# write the calculate chunk of the field 
outputIteration.meshes['E']['x'].store_chunk(ex.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
outputIteration.meshes['E']['y'].store_chunk(ey.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
outputIteration.meshes['E']['z'].store_chunk(ez.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
outputIteration.meshes['B']['x'].store_chunk(bx.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
outputIteration.meshes['B']['y'].store_chunk(by.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
outputIteration.meshes['B']['z'].store_chunk(bz.reshape(xdim, ydim, zdim), chunk_offset, chunk_size)
writeSeries.flush()

## When finished with all field chunks
you can write the other data to the file

In [ ]:

write()
print("data written")


# Visualization of the fields if necessary

and other stuff, not realy documented with comments

## absolute field values

In [ ]:
# absolute values of the fields
ef = np.sqrt(ex**2+ey**2+ez**2)
bf = np.sqrt(bx**2+by**2+bz**2)

In [ ]:

depth = 400
plt.imshow((ef.reshape(zdim, ydim, xdim)[depth].T))
plt.colorbar()
plt.show()

## show particle positions

In [ ]:
figsize(15,15)
a = np.histogram2d(xpos, ypos, weights=weightings, bins=[np.linspace(0, xdim, xdim+1), np.linspace(0, ydim, ydim+1)] )
imshow(a[0])
xlabel("y cells")
ylabel("x cells")
show()

In [ ]:
del ex_d
del ey_d
del ez_d
del bx_d
del by_d
del bz_d

In [ ]:
del writeSeries

In [ ]:
fields

In [ ]:
v = 5e-4
figure = plt.figure(figsize=(10,10))
plt.imshow(fields['B']['z'].reshape(768, 512, 768)[500].T, vmin=-v, vmax=v)
plt.colorbar()